# Normalização de Dados

- Nesse Script os dados serão normalizados e re-escritos em uma zona do datalake chamada Trusted;
- A normalização deve ser feita para todos os arquivos contidos;

In [2]:
import pandas as pd

In [ ]:
var_path =  'C:/Users/luizf/OneDrive/Documentos/temp/'

df_items = pd.read_csv(var_path + 'olist_order_items_dataset.csv')
df_reviews = pd.read_csv(var_path + 'olist_order_reviews_dataset.csv')
df_orders = pd.read_csv(var_path + 'olist_orders_dataset.csv')
df_products = pd.read_csv(var_path + 'olist_products_dataset.csv')
df_geolocation = pd.read_csv(var_path + 'olist_geolocation_dataset.csv')
df_sellers = pd.read_csv(var_path + 'olist_sellers_dataset.csv')
df_payments = pd.read_csv(var_path + 'olist_order_payments_dataset.csv')
df_customers = pd.read_csv(var_path + 'olist_customers_dataset.csv')
df_category = pd.read_csv(var_path + 'product_category_name_translation.csv')

- Converter todos os tipos de data em timestamp

### Items

In [4]:
df_items.head(5) 

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [5]:
test = df_items.head(5)

In [45]:
df_items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [7]:
df_items["shipping_limit_date"] = pd.to_datetime(df_items["shipping_limit_date"]) 

### Reviews

In [46]:
df_reviews.dtypes

review_id                          object
order_id                           object
review_score                        int64
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
review_score_classific             object
sentimento                         object
dtype: object

In [15]:
# Tratando casos
trocas = {
    1 : "Péssimo",
    2 : "Abixo da média",
    3 : "Na média",
    4 : "Acima da média",
    5 : "Ótimo"
}

# Realizando a troca dos campos
df_reviews['review_score_classific'] = df_reviews['review_score'].map(trocas)

In [16]:
# Tratando casos
trocas = {
    1 : "negativo",
    2 : "negativo",
    3 : "neutro",
    4 : "positivo",
    5 : "positivo"
}

# Realizando a troca dos campos
df_reviews['sentimento'] = df_reviews['review_score'].map(trocas)

In [17]:
df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"]) 
df_reviews["review_answer_timestamp"] = pd.to_datetime(df_reviews["review_answer_timestamp"]) 

In [19]:
df_ava_not_null_and_not_neutral = df_reviews[(df_reviews['sentimento'].isin(['positivo','negativo','neutro'])) & (~(df_reviews['review_comment_message'].isnull())) ]

In [20]:
df_ava_not_null_and_not_neutral

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_score_classific,sentimento
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,Ótimo,positivo
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,Ótimo,positivo
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22,2018-05-23 16:45:47,Acima da média,positivo
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16,2018-02-20 10:52:22,Acima da média,positivo
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23,2018-05-24 03:00:01,Ótimo,positivo
...,...,...,...,...,...,...,...,...,...
99205,98fffa80dc9acbde7388bef1600f3b15,d398e9c82363c12527f71801bf0e6100,4,NaN,para este produto recebi de acordo com a compr...,2017-11-29,2017-11-30 15:52:51,Acima da média,positivo
99208,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07,2018-02-19 19:47:23,Ótimo,positivo
99215,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19,2018-05-20 21:51:06,Na média,neutro
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43,Ótimo,positivo


### Orders

In [49]:
df_orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [24]:
df_orders["order_purchase_timestamp"] = pd.to_datetime(df_orders["order_purchase_timestamp"]) 
df_orders["order_approved_at"] = pd.to_datetime(df_orders["order_approved_at"]) 
df_orders["order_delivered_carrier_date"] = pd.to_datetime(df_orders["order_delivered_carrier_date"]) 
df_orders["order_estimated_delivery_date"] = pd.to_datetime(df_orders["order_estimated_delivery_date"]) 
df_orders["order_delivered_customer_date"] = pd.to_datetime(df_orders["order_delivered_customer_date"]) 


### Products 

In [27]:
df_products.drop_duplicates()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


### Sellers

In [32]:
df_sellers.shape

(3095, 4)

In [29]:
df_sellers.dtypes

seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object

In [31]:
df_sellers.drop_duplicates()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


### Payments

In [37]:
df_payments = df_payments.drop_duplicates()

In [39]:
df_payments.shape

(103886, 5)

In [33]:
df_payments.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

### customers

In [40]:
df_customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

### Categoria


In [41]:
df_category.dtypes

product_category_name            object
product_category_name_english    object
dtype: object

### Final

In [51]:
df_reviews.dtypes

review_id                          object
order_id                           object
review_score                        int64
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
review_score_classific             object
sentimento                         object
dtype: object

In [50]:
df_reviews
df_refiews_final = df_ava_not_null_and_not_neutral
df_orders
df_products
df_geolocation

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [28]:
df_geolocation

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [9]:
def max_digitos_float(df, coluna):
    """
    Retorna o número máximo de dígitos antes e depois da vírgula
    de uma coluna float de um DataFrame.
    """
    # Remove nulos e converte para string
    col_str = df[coluna].dropna().map(lambda x: format(x, 'f'))

    # Separa parte inteira e decimal
    partes = col_str.str.split('.', expand=True)

    # Máximo de dígitos antes e depois da vírgula
    max_inteira = partes[0].str.len().astype(int).max()
    max_decimal = (
        partes[1].str.rstrip('0').str.len().astype(int).max()
        if 1 in partes.columns else 0
    )

    return f"({max_inteira}, {max_decimal})"

In [10]:
def max_digitos_str(df, coluna):
    aux = df[coluna].astype(str).str.len().max()

    return f'({aux})'

In [11]:
max_digitos_str(test, "order_item_id")

'(1)'

In [12]:
max_digitos_float(test, "price")

'(3, 2)'

In [13]:
test.columns[0]

'order_id'

In [14]:
tamanho_max_order_id = df_items['order_id'].astype(str).str.len().max()
tamanho_max_order_item_id = df_items['order_id'].astype(str).str.len().max()
tamanho_max_order_product_id = df_items['order_id'].astype(str).str.len().max()
tamanho_max_seller_id = df_items['order_id'].astype(str).str.len().max()

tamanho_max_order_id = df_items['order_id'].astype(str).str.len().max()


In [52]:
df_items.to_csv('itens_trusted.csv')
df_refiews_final.to_csv('reviews_trusted.csv')
df_orders.to_csv('order_trusted.csv')
df_products.to_csv('products_trusted.csv')
df_geolocation.to_csv('geolocation_trusted.csv')
df_sellers.to_csv('sellers_trusted.csv')
df_payments.to_csv('paymentas_trusted.csv')
df_customers.to_csv('customers_trusted.csv')
df_category.to_csv('category_trusted.csv')

In [ ]:
df_items = pd.read_csv(var_path + 'olist_order_items_dataset.csv')
df_reviews = pd.read_csv(var_path + 'olist_order_reviews_dataset.csv')
df_orders = pd.read_csv(var_path + 'olist_orders_dataset.csv')
df_products = pd.read_csv(var_path + 'olist_products_dataset.csv')
df_geolocation = pd.read_csv(var_path + 'olist_geolocation_dataset.csv')
df_sellers = pd.read_csv(var_path + 'olist_sellers_dataset.csv')
df_payments = pd.read_csv(var_path + 'olist_order_payments_dataset.csv')
df_customers = pd.read_csv(var_path + 'olist_customers_dataset.csv')
df_category = pd.read_csv(var_path + 'product_category_name_translation.csv')